# EXPLORATION 4 : 작사가 인공지능 만들기

## 준비한 데이터
- lyrics dataset

## 1. 데이터 읽어오기

In [1]:
import tensorflow as tf
import glob
import os
import re         # 정규표현식을 위한 모듈 (문장 데이터를 정돈) 
import numpy as np

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:                  # 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free', 'They come to join us here', 'From sea to shining sea And they all have a dream', 'As people always will', 'To be safe and warm', 'In that shining city on the hill Some wanna slam the door', 'Instead of opening the gate', "Aw, let's turn this thing around"]


## 2. 데이터 정제하기

In [2]:
def preprocess_sentence(sentence):
        
    sentence = sentence.lower().strip()                       # 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)       # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence)               # 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)     # aa-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip()                               # 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>'              # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 문장이 필터링되는지 확인하기

<start> this is sample sentence . <end>


In [3]:
corpus = []

for sentence in raw_corpus:                 # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue        
    if sentence[-1] == ":": continue
    if sentence[0] == "[" : continue
    if sentence.count(" ") > 10 : continue  

    corpus.append(preprocess_sentence(sentence))       # 정제를 하고 담아주세요
        
len(corpus)       # 몇개가 담겼는지 확인하기

150691

## 3. tokenize 패키기 사용하기

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(              
        num_words=20000,                                            # 전체 단어의 개수 
        filters=' ',                                                # 별도로 전처리 로직을 추가가능
        oov_token="<unk>"                                           # out-of-vocabulary
    )
    tokenizer.fit_on_texts(corpus)                                  # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   43   64 ...    0    0    0]
 [   2    9 2958 ...    0    0    0]
 [   2   43   64 ...    0    0    0]
 ...
 [   2  555   20 ...    0    0    0]
 [   2  121   36 ...    0    0    0]
 [   2    5   23 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f247f0d9d90>


In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


## 4. 텐서 데이터 확인하기

In [5]:
tensor.shape         

(150691, 15)

In [6]:
print(tensor[:10, :])

[[   2   43   64   73  704    3    0    0    0    0    0    0    0    0
     0]
 [   2    9 2958   10   27  262    3    0    0    0    0    0    0    0
     0]
 [   2   43   64   10 2124  126   92    3    0    0    0    0    0    0
     0]
 [   2   73  544   10 1070  544    8   43   25   76    9  345    3    0
     0]
 [   2   82  173  171   84    3    0    0    0    0    0    0    0    0
     0]
 [   2   10   27 1061    8  903    3    0    0    0    0    0    0    0
     0]
 [   2 1031   21 5583    6 2959    3    0    0    0    0    0    0    0
     0]
 [   2 1571    4   66   16  192   44  185  134    3    0    0    0    0
     0]
 [   2  176   11  620  101  400  337    3    0    0    0    0    0    0
     0]
 [   2   11   16   29   10   12    8    7    3    0    0    0    0    0
     0]]


In [7]:
len(tensor[0])

15

In [9]:
src_input = tensor[:, :-1]   # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  43  64  73 704   3   0   0   0   0   0   0   0   0]
[ 43  64  73 704   3   0   0   0   0   0   0   0   0   0]


## 5. 평가 데이터셋 분리하기

In [10]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size=0.2,
                                                         shuffle=True)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (120552, 14)
Target Train: (120552, 14)


## 6. 훈련데이터셋 생성하기

In [11]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 512
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((512, 14), (512, 14)), types: (tf.int32, tf.int32)>

## 7. 텍스트 제너레이터 모델 생성하기

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 3072
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(512, 14, 20001), dtype=float32, numpy=
array([[[ 1.7306811e-04,  2.7666694e-05, -4.0732011e-05, ...,
          1.0831895e-04,  4.6472007e-05,  1.9541067e-04],
        [ 2.6668518e-04,  4.4235499e-06, -1.4309929e-04, ...,
          4.7978610e-04,  7.6409582e-05,  3.3636941e-04],
        [ 3.5137465e-04,  2.1702774e-04, -4.3320609e-04, ...,
          6.6236535e-04, -3.1764564e-05,  6.3351175e-04],
        ...,
        [-1.7411254e-03,  1.1870423e-03,  3.9721778e-04, ...,
         -1.4192265e-04, -1.1192209e-03,  2.9676426e-03],
        [-1.9899332e-03,  1.4661695e-03,  5.4759311e-04, ...,
         -2.4668136e-04, -1.1746883e-03,  3.2117842e-03],
        [-2.1979348e-03,  1.7152413e-03,  6.9151731e-04, ...,
         -3.4399095e-04, -1.2078827e-03,  3.4064411e-03]],

       [[ 1.7306811e-04,  2.7666694e-05, -4.0732011e-05, ...,
          1.0831895e-04,  4.6472007e-05,  1.9541067e-04],
        [ 3.4548674e-04,  4.9869399e-05,  4.5183455e-04, ...,
          2.2205610e-04, 

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  10240512  
_________________________________________________________________
lstm (LSTM)                  multiple                  44052480  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  75509760  
_________________________________________________________________
dense (Dense)                multiple                  61463073  
Total params: 191,265,825
Trainable params: 191,265,825
Non-trainable params: 0
_________________________________________________________________


## 8. 모델훈련하기

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
235/235 [==============================] - 926s 4s/step - loss: 4.3343
Epoch 2/10
235/235 [==============================] - 933s 4s/step - loss: 3.1286
Epoch 3/10
235/235 [==============================] - 935s 4s/step - loss: 2.8960
Epoch 4/10
235/235 [==============================] - 935s 4s/step - loss: 2.7308
Epoch 5/10
235/235 [==============================] - 935s 4s/step - loss: 2.5723
Epoch 6/10
235/235 [==============================] - 935s 4s/step - loss: 2.4203
Epoch 7/10
235/235 [==============================] - 934s 4s/step - loss: 2.2823
Epoch 8/10
235/235 [==============================] - 933s 4s/step - loss: 2.1402
Epoch 9/10
235/235 [==============================] - 932s 4s/step - loss: 2.0247
Epoch 10/10
235/235 [==============================] - 932s 4s/step - loss: 1.8967


## 9. 테스트 데이터 셋 생성하기

In [16]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 512
steps_per_epoch = len(enc_val) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개

test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset

<BatchDataset shapes: ((512, 14), (512, 14)), types: (tf.int32, tf.int32)>

## 10. 평가하기

In [17]:
results=model.evaluate(test_dataset)

58/58 [==============================] - 79s 1s/step - loss: 2.4285


## 11. 문장생성기 구축하기

In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

## 12. 문장 생성해보기

In [19]:
generate_text(model, tokenizer, init_sentence="<start> you")

'<start> you know i m bad , i m bad shameone <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> i")

'<start> i m a survivor <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> can")

'<start> can t be now <end> '

# 프로젝트 결과
### 느낀점
1. hidden_size = 3072 , embedding_size = 512 로 바꿔서 진행하니까 토탈 파라미터의 수가 1억 9천개가 넘어갔고, 한 에폭당 15분이 넘도록 걸려서 너무 오래 기다렸지만 결과적으로 문장이 잘 생성되어서 다행이었다.
2. 코드 하나하나의 의미를 분석해보려고 최대한 노력해보았는데 자연어처리는 처음이어서 어려운 부분이 많았다. RNN에 대해 공부의 필요성을 느꼈다.
3. 정규표현식이 자연어처리에서 필수적이라는 생각이 들었고 자연어처리뿐만 아니라 다양한 곳에서 필요하기 때문에 한번 날을 잡아서 공부해야겠다는 생각이 들었다.
4. 토크나이즈, 텐서에 대해 공부를 하면서 자연어처리의 흐름에 대해 좀 더 알 수 있어서 좋았다.